# Aprendizado de Máquina
**Aluna**: _Bruna Guedes Pereira, Laura Medeiros Dal Ponte, Mariana Melo Pereira_

## Tentando algo diferente

_Modelos 4_

### Introdução

O aprendizado de máquina consolidou-se como uma das ferramentas mais influentes da ciência contemporânea, permitindo que dados complexos promovam decisões automatizadas. Dentro desse campo, os algoritmos de classificação ocupam papel central, pois possibilitam identificar padrões e organizar informações em categorias de forma precisa e eficiente.

Métodos clássicos como Support Vector Machine (SVM), Naive Bayes e Discriminant Analysis possuem relevância histórica na resolução de problemas de classificação, sendo ainda muito utilizados em diversos experimentos científicos. Explorar o funcionamento interno desses algoritmos, compreendendo suas bases matemáticas e avaliando seu desempenho em conjuntos de dados reais, é um exercício essencial de formação acadêmica.

Dessa forma, o presente trabalho dedica-se a investigar o algoritmos ------------, implementá-lo de maneira didática e comparar seus resultados a um modelo de referência.

### Discussões e conclusão

### Referências Bibliográficas
1. IBM. Máquina de vetores de suporte. Disponível em: <https://www.ibm.com/br-pt/think/topics/support-vector-machine>.
2. OPENAI. ChatGPT. São Francisco: OpenAI, 2025. Disponível em: <https://chat.openai.com/>. Acesso em: 10 set. 2025.